<a href="https://colab.research.google.com/github/Asigen93/voicecloning/blob/main/tes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setup Environment**

In [ ]:
!python --version

In [3]:
!sudo apt-get update -y
!sudo apt-get install python3.10 python3.10-distutils python3.10-venv -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 2
!sudo update-alternatives --config python3

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,014 kB]
Get:4 https://cli.github.com/packages stable InRelease [3,917 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,305 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/mai

In [4]:
!python --version

Python 3.10.12


In [6]:
!wget https://bootstrap.pypa.io/get-pip.py
!python3 get-pip.py

--2025-09-30 04:08:50--  https://bootstrap.pypa.io/get-pip.py
Resolving bootstrap.pypa.io (bootstrap.pypa.io)... 151.101.0.175, 151.101.64.175, 151.101.128.175, ...
Connecting to bootstrap.pypa.io (bootstrap.pypa.io)|151.101.0.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2148718 (2.0M) [text/x-python]
Saving to: ‘get-pip.py’

get-pip.py          100%[===================>]   2.05M  --.-KB/s    in 0.1s    

2025-09-30 04:08:50 (17.2 MB/s) - ‘get-pip.py’ saved [2148718/2148718]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pip]


In [7]:
!pip --version

pip 25.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [11]:
!pip install TTS==0.22.0 unidecode phonemizer --ignore-installed blinker

  Using cached TTS-0.22.0-cp310-cp310-manylinux1_x86_64.whl.metadata (21 kB)
  Using cached Unidecode-1.4.0-py3-none-any.whl.metadata (13 kB)
  Using cached phonemizer-3.3.0-py3-none-any.whl.metadata (48 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached cython-3.1.4-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (5.0 kB)
  Using cached scipy-1.15.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached torch-2.8.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached torchaudio-2.8.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (7.2 kB)
  Using cached soundfile-0.13.1-py2.py3-none-manylinux_2_28_x86_64.whl.metadata (16 kB)
  Using cached librosa-0.11.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached scikit_learn-1.7.2-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached inflect-7.5.0-py3-none-any.whl.metadata (24 kB)
 

In [14]:
import os
from IPython.display import Audio

# **Persiapan Dataset**

In [15]:
!mkdir -p /content/dataset/wavs

# **Preprocessing Audio**

In [16]:
import librosa, soundfile as sf
input_dir = "/content/dataset/wavs"
output_dir = "/content/dataset/wavs_proc"
os.makedirs(output_dir, exist_ok=True)

for file in os.listdir(input_dir):
    if file.endswith(".wav"):
        y, sr = librosa.load(os.path.join(input_dir, file), sr=22050, mono=True)
        y = y / max(abs(y))
        sf.write(os.path.join(output_dir, file), y, 22050)

print("Audio siap dipakai untuk training.")

Audio siap dipakai untuk training.


# **Konfigurasi Fine-tuning**

In [ ]:
config_path = "/content/config.json"
config_json = """
{
  "model": "tts_models/multilingual/multi-dataset/your_tts",
  "run_name": "finetune_my_voice",
  "output_path": "/content/finetuned_model",
  "datasets": [
      {
        "formatter": "ljspeech",
        "path": "/content/dataset",
        "meta_file_train": "metadata.csv",
        "meta_file_val": "metadata.csv"
      }
    ],
    "trainer": {
        "epochs": 50,
        "batch_size": 16,
        "eval_batch_size": 16
    }
}
"""
with open(config_path, "w") as f:
    f.write(config_json)

# **Training Fine-tuning**

In [ ]:
from TTS.bin.train import main as tts_train
!tts_train --config_path /content/config.json

# **Inference (Testing)**

In [ ]:
from TTS.api import TTS
model_path = "/content/finetuned_model/best_model.pth"
config_path = "/content/finetuned_model/config.json"
tts = TTS(model_path, config_path)

text = "halo, ini adalah hasil cloning suara untuk skripsi saya"
output_file = "output.wav"
tts.tts_to_file(text=text, file_path=output_file)
Audio(output_file)